# CIFAR nvsmi traces

In [10]:
%matplotlib inline
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os.path
import subprocess
import datetime
from cycler import cycler
import pandas as pd

In [5]:
!ls logs/combined_profiles

cifar_log_b100_l0.150_00.log       cifar_log_b505_l0.150_01.nvsmi.csv
cifar_log_b100_l0.150_00.nvsmi.csv cifar_log_b507_l0.150_01.log
cifar_log_b102_l0.150_00.log       cifar_log_b507_l0.150_01.nvsmi.csv
cifar_log_b102_l0.150_00.nvsmi.csv cifar_log_b509_l0.150_00.log
cifar_log_b103_l0.150_00.log       cifar_log_b509_l0.150_00.nvsmi.csv
cifar_log_b103_l0.150_00.nvsmi.csv cifar_log_b509_l0.150_01.log
cifar_log_b104_l0.150_00.log       cifar_log_b509_l0.150_01.nvsmi.csv
cifar_log_b104_l0.150_00.nvsmi.csv cifar_log_b510_l0.150_00.log
cifar_log_b105_l0.150_00.log       cifar_log_b510_l0.150_00.nvsmi.csv
cifar_log_b105_l0.150_00.nvsmi.csv cifar_log_b510_l0.150_01.log
cifar_log_b106_l0.150_00.log       cifar_log_b510_l0.150_01.nvsmi.csv
cifar_log_b106_l0.150_00.nvsmi.csv cifar_log_b512_l0.150_00.log
cifar_log_b107_l0.150_00.log       cifar_log_b512_l0.150_00.nvsmi.csv
cifar_log_b107_l0.150_00.nvsmi.csv cifar_log_b512_l0.150_01.log
cifar_log_b108_l0.150_00.log       cifar_log_b512_l0.150_01.nv

In [7]:
batch_learn_pattern = re.compile(r"^b\s*(\d+)\s+l\s*([0-9\.]+)")
output_pattern = re.compile(r"^(\d+)\s+([0-9\.]+)\s+([0-9\.]+)\s+([0-9\.]+)\s+([0-9\.]+)\s+([0-9\.]+)")
remove_str="[J"


# Read minibatch size and 1st epoch time from files.
# Store in a DataFrame.
def fileToDF(logfile, debug=False):
    global batch_learn_pattern
    global output_pattern
    global remove_str
    logfile = logfile.strip(" \n")
    with open(logfile,"r") as f:
        lines = f.readlines()
        batch = 0
        time = 0
        epoch = 0
        ind = 0 # DataFrame row numebr (index)
        df = pd.DataFrame(data=None,columns=["batch","learn","epoch","time"])
        for line in lines:
            s = line.strip(' \n')
            s = s.replace(remove_str,"")
            if debug: print s,
            m = batch_learn_pattern.match(s)
            if m:
                batch = int(m.group(1))
                learn = float(m.group(2))
                if debug: print "BS,LR:",batch,learn
            else:
                m2 = output_pattern.match(s)
                if m2:
                    epoch = int(m2.group(1))
                    time = float(m2.group(6))
                    if debug: print "BS,LR,epoch,time:",batch,learn,epoch,time
                    df.loc[ind] = [batch,learn,epoch,time]
                    ind += 1
                    
    return df

In [45]:
# Parse date from readable format to seconds
def parseTime(date_time,start=0):
    if isinstance(date_time,float):
        print date_time,"is float"
        return date_time
    dt_obj = datetime.datetime.strptime(date_time, "%Y/%m/%d %H:%M:%S.%f")
    seconds = 0
    if start == 0:
        seconds = dt_obj
    else:
        seconds = (dt_obj - start).total_seconds()
    return seconds

In [52]:
logdir = "logs/combined_profiles"
batch_learn_pattern = re.compile(r"^b\s*(\d+)\s+l\s*([0-9\.]+)")
batch_learn_run_pattern = re.compile(r"_b(\d+)_l([0-9\.])+_(\d+)") # Match against file name
output_pattern = re.compile(r"^(\d+)\s+([0-9\.]+)\s+([0-9\.]+)\s+([0-9\.]+)\s+([0-9\.]+)\s+([0-9\.]+)")
remove_str="[J"
log_filename_pattern=re.compile(r"cifar_.*\.log")
smi_filename_pattern=re.compile(r"cifar_.*\.csv")

def readLogs(logdir, debug=False):
    global log_filename_pattern
    global smi_filename_pattern
    list_command = "ls -1 "+logdir
    log_files=[]
    smi_files=[]
    proc = subprocess.Popen(list_command.split(" "),
                         stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
    maxfiles = 5
    for line in iter(proc.stdout.readline,b''):
        line = line.strip(" \n")
        m = log_filename_pattern.match(line)
        if m:
            log_files.append(os.path.abspath(os.path.join(logdir,line)))
        else:
            m1 = smi_filename_pattern.match(line)
            if m1:
                smi_files.append(os.path.abspath(os.path.join(logdir,line)))
            
    #if debug: print "files:",files
    df = pd.DataFrame(data=None,columns=["batch","learn","epoch","time"])

    filecounter=0
    for file in log_files:
        if debug: print file
        df1 = fileToDF(file,debug)
        if len(df1) > 0:
            df = pd.concat([df,df1],ignore_index=True)
        filecounter +=1
        if filecounter>=maxfiles:
            break
            
    print "========="
    filecounter=0
    smi_df = pd.DataFrame(data=None)
    for file in smi_files:
        if debug: print file
        bs = 0
        lr = 0
        run = 0
        m = batch_learn_run_pattern.search(file)
        if m:
            bs = int(m.group(1))
            lr = float(m.group(2))
            run = int(m.group(3))
        df1 = pd.read_csv(file)
        df1["bs"] = bs
        df1["lr"] = lr
        df1["run"] = run
        # Pase time column
        start_time = parseTime(df1["timestamp"][0])
        print "Start time:",start_time
        df1["sec"] = df1["timestamp"].apply(lambda x: parseTime(x,start_time))
        
        
        if len(smi_df.index) < 1:
            smi_df = df1
        else:
            smi_df = pd.concat([smi_df,df1],ignore_index=True)
        filecounter +=1
        if filecounter>=maxfiles:
            break
    return (df, smi_df)

In [53]:
df,df1 = readLogs(logdir,True)

/Users/peterbryzgalov/work/ML/mlbenchmarks/Chainer/logs/combined_profiles/cifar_log_b100_l0.150_00.log
b100 l0.15 BS,LR: 100 0.15
GPU: 0 # Minibatch-size: 100 # epoch: 1  Using CIFAR100 dataset. epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time 1           4.57676     4.34638               0.0199401      0.0302                    90.8268 BS,LR,epoch,time: 100 0.15 1 90.8268
/Users/peterbryzgalov/work/ML/mlbenchmarks/Chainer/logs/combined_profiles/cifar_log_b102_l0.150_00.log
b102 l0.15 BS,LR: 102 0.15
GPU: 0 # Minibatch-size: 102 # epoch: 1  Using CIFAR100 dataset. epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time 1           4.68291     4.51105               0.0132982      0.0189146                 90.7188 BS,LR,epoch,time: 102 0.15 1 90.7188
/Users/peterbryzgalov/work/ML/mlbenchmarks/Chainer/logs/combined_profiles/cifar_log_b103_l0.150_00.log
b103 l0.15 BS,LR: 103 0.15
GPU: 0 # Minibat

In [54]:
df1[df1["bs"]==104].head()

,timestamp,name,memory.total [MiB],memory.used [MiB],utilization.gpu [%],utilization.memory [%],bs,lr,run,sec
5750,2018/04/14 00:36:28.363,Quadro P2000,5053 MiB,882 MiB,0 %,1 %,104,0.0,0,0.000
5751,2018/04/14 00:36:28.413,Quadro P2000,5053 MiB,882 MiB,0 %,1 %,104,0.0,0,0.050
5752,2018/04/14 00:36:28.464,Quadro P2000,5053 MiB,882 MiB,0 %,1 %,104,0.0,0,0.101
5753,2018/04/14 00:36:28.514,Quadro P2000,5053 MiB,882 MiB,0 %,1 %,104,0.0,0,0.151
5754,2018/04/14 00:36:28.564,Quadro P2000,5053 MiB,882 MiB,0 %,1 %,104,0.0,0,0.201


In [55]:
df1.drop(["timestamp"],axis=1,inplace=True)
df1.head()

,name,memory.total [MiB],memory.used [MiB],utilization.gpu [%],utilization.memory [%],bs,lr,run,sec
0,Quadro P2000,5053 MiB,882 MiB,0 %,1 %,100,0.0,0,0.000
1,Quadro P2000,5053 MiB,882 MiB,0 %,1 %,100,0.0,0,0.050
2,Quadro P2000,5053 MiB,882 MiB,0 %,1 %,100,0.0,0,0.101
3,Quadro P2000,5053 MiB,882 MiB,0 %,1 %,100,0.0,0,0.151
4,Quadro P2000,5053 MiB,882 MiB,0 %,1 %,100,0.0,0,0.201


In [23]:
df.head()

,batch,learn,epoch,time
0,100.0,0.15,1.0,90.8268
1,102.0,0.15,1.0,90.7188
2,103.0,0.15,1.0,90.8106
3,104.0,0.15,1.0,90.3553
4,105.0,0.15,1.0,90.7617
